In [1]:
import os
import json
import h5py
import numpy as np

import robomimic
import robomimic.utils.file_utils as FileUtils
import robomimic.utils.env_utils as EnvUtils
import robomimic.utils.obs_utils as ObsUtils
import imageio
import tqdm
from robomimic.utils.file_utils import create_hdf5_filter_key

In [2]:
# dataset_path = "/home/ns1254/robomimic/datasets/square/mh/image_v141.hdf5"
dataset_path = "/home/ns1254/robomimic/datasets/can/mh/image_v141.hdf5"
f_org = h5py.File(dataset_path, "r")
demos = list(f_org["data"].keys())

In [3]:
lengths=[]
for demo_name in demos:
    demo=f_org['data'][demo_name]
    num_samples=demo.attrs['num_samples']
    lengths.append(num_samples)

lengths=np.array(lengths)

print('Number of demos: ', len(demos))
print('Max length: ', np.max(lengths))
print('Min length: ', np.min(lengths))
print('Mean length: ', np.mean(lengths))

Number of demos:  300
Max length:  1050
Min length:  98
Mean length:  209.18666666666667


In [4]:
f_org['mask'].keys()

<KeysViewHDF5 ['20_percent', '20_percent_train', '20_percent_valid', '50_percent', '50_percent_train', '50_percent_valid', 'better', 'better_operator_1', 'better_operator_1_train', 'better_operator_1_valid', 'better_operator_2', 'better_operator_2_train', 'better_operator_2_valid', 'better_train', 'better_valid', 'okay', 'okay_better', 'okay_better_train', 'okay_better_valid', 'okay_operator_1', 'okay_operator_1_train', 'okay_operator_1_valid', 'okay_operator_2', 'okay_operator_2_train', 'okay_operator_2_valid', 'okay_train', 'okay_valid', 'train', 'valid', 'worse', 'worse_better', 'worse_better_train', 'worse_better_valid', 'worse_okay', 'worse_okay_train', 'worse_okay_valid', 'worse_operator_1', 'worse_operator_1_train', 'worse_operator_1_valid', 'worse_operator_2', 'worse_operator_2_train', 'worse_operator_2_valid', 'worse_train', 'worse_valid']>

In [5]:
demos[:3]

['demo_0', 'demo_1', 'demo_10']

In [6]:
good_demos1=[demo.decode('utf-8') for demo in f_org['mask']['better_operator_1']]
good_demos2=[demo.decode('utf-8') for demo in f_org['mask']['better_operator_2']]
len(good_demos1), len(good_demos2)

(50, 50)

In [7]:
good_demos=good_demos1#+good_demos2
good_demos[45:55]

['demo_5', 'demo_6', 'demo_7', 'demo_8', 'demo_9']

In [8]:
len(demos)

300

In [9]:
# f_org.close()

In [10]:
# demos2copy=[demo.decode('utf-8') for demo in f_org['mask']['expert']] 
demos2copy=good_demos  
len(demos2copy)

50

In [11]:
demos2copy[:5]

['demo_0', 'demo_1', 'demo_10', 'demo_11', 'demo_12']

In [12]:
# fn='/home/ns/collect_robomimic_demos/PickPlaceCan_02_09_2024_04_37PM_sojib/videos/expert/expert2.txt'
# with open(fn, 'r') as f:
#     file_demos = f.readlines()
# demos = [x.strip().replace(".mp4","").strip() for x in file_demos]
# len(demos), demos[:5]

In [13]:
dataset_path_sub =dataset_path
dataset_path_sub=dataset_path_sub.replace(".hdf5", "")+"_sub.hdf5"
dataset_path_sub

'/home/ns1254/robomimic/datasets/can/mh/image_v141_sub.hdf5'

In [14]:
f_sub = h5py.File(dataset_path_sub, "w")

In [15]:
keys_to_copy=demos2copy
len(keys_to_copy)

50

In [16]:
f_sub.create_group("data")
f_sub.create_group("mask")

<HDF5 group "/mask" (0 members)>

In [17]:
attrs=f_org["data"].attrs["env_args"]
f_sub["data"].attrs["env_args"] = attrs

In [18]:
i=0
demos=[]
for demo_name in tqdm.tqdm(keys_to_copy):
    b_demo=f_org['data'][demo_name]
    num_samples=b_demo.attrs['num_samples']
    model_file=b_demo.attrs['model_file']

    bdemo_name="demo_"+str(i)
    demos.append(bdemo_name)
    i = i+1

    f_sub["data"].create_group(bdemo_name)


    ks=['actions', 'dones', 'rewards', 'states']
    for k in ks:
        kdata = b_demo[k]
        f_sub["data"][bdemo_name].create_dataset(k, data=kdata)

    obs_keys =b_demo['obs'].keys()
    next_obs_keys=b_demo['next_obs'].keys()
    for k in obs_keys:
        kdata = b_demo['obs'][k]
        f_sub["data"][bdemo_name].create_dataset(f'obs/{k}', data=kdata)
    for k in next_obs_keys:
        kdata = b_demo['next_obs'][k]
        f_sub["data"][bdemo_name].create_dataset(f'next_obs/{k}', data=kdata)

    f_sub["data"][bdemo_name].attrs["num_samples"] = num_samples
    f_sub["data"][bdemo_name].attrs["model_file"] = model_file
 
# f_sub.close()
# f_org.close()

100%|██████████| 50/50 [00:04<00:00, 11.65it/s]


In [19]:
# demos

In [20]:
# demos=demos2copy
demos= np.array(demos, dtype='S16') 
demos.shape, demos[:4]

((50,), array([b'demo_0', b'demo_1', b'demo_2', b'demo_3'], dtype='|S16'))

In [21]:
hdf5_path=dataset_path_sub 

filter_keys=sorted([elem for elem in demos])
filter_name="better_operator_1"
filter_lengths = create_hdf5_filter_key(hdf5_path=hdf5_path, demo_keys=filter_keys, key_name=filter_name)

In [22]:
f_sub.close()
f_org.close()

In [22]:
i

100